In [1]:
from sklearn.datasets import fetch_openml

In [2]:
mnist = fetch_openml('mnist_784', version=1)

In [3]:
X, y = mnist['data'], mnist['target']
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [20]:
from scipy.ndimage.interpolation import shift
import pandas as pd

In [ ]:
def gen_shifted_df(seq):
    shifted_arr = shift(image, seq).reshape(784,1).T
    return pd.DataFrame(shifted_arr, columns=X_train.columns)

for i in range(len(X_train)):
    image = X_train.iloc[i].values.reshape(28,28)
    left_df = gen_shifted_df([0,-1])
    right_df = gen_shifted_df([0, 1])
    up_df = gen_shifted_df([-1, 0])
    down_df = gen_shifted_df([1, 0])
    X_train = pd.concat([X_train, left_df, right_df,
                         up_df, down_df])
    labels = [pd.Series(y_train.iloc[i]) for _ in range(4)]
    y_train = pd.concat([y_train] + labels)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=4,
                               weights='distance')
knn_clf.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
y_test_pred = cross_val_predict(knn_clf, X_test, y_test,
                                 cv=3,
                                 verbose=2)

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
precision_score(y_test, y_test_pred, average='macro')